In [ ]:
# Cycle Consitency Model

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
'''

Input: x (PPG 시계열 데이터, shape: [B, 1, 2100])
      │
      ▼
[Encoder]
      │
      ├── enc1 → s1: Conv1D → ReLU → ResidualBlock
      │         (shape: [B, 32, 2100])
      ├── pool1 → (shape: [B, 32, 1050])
      │
      ├── enc2 → s2: Conv1D → ReLU → ResidualBlock
      │         (shape: [B, 64, 1050])
      ├── pool2 → (shape: [B, 64, 525])
      │
      ├── enc3 → s3: Conv1D → ReLU → ResidualBlock
      │                    SelfAttentionBlock
      │         (shape: [B, 128, 525])
      └── pool3 → z (shape: [B, 128, 262])


[Decoder A (gen_A)]
      ├── up1: ConvTranspose1D (↑2) → [B, 128, 524]
      ├── concat with center_crop(s3) → [B, 256, 524]
      ├── dec1: Conv1D → ReLU → ResidualBlock → [B,128]
      ├── up2: ConvTranspose1D (↑2) → [B, 64, 1048]
      ├── concat with center_crop(s2) → [B, 128, 1048]
      ├── dec2: Conv1D → ReLU → ResidualBlock → [B,64]
      ├── up3: ConvTranspose1D (↑2) → [B, 32, 2096]
      ├── concat with center_crop(s1) → [B, 64, 2096]
      ├── dec3: Conv1D → ReLU → ResidualBlock → [B,32]
      └── final: Conv1D(→1) → x' (x_prime) [B, 1, 2096]
  '''

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

# --------- Residual Block ---------
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv1d(channels, channels, 3, padding=1),
            nn.BatchNorm1d(channels),
            nn.ReLU(),
            nn.Conv1d(channels, channels, 3, padding=1),
            nn.BatchNorm1d(channels)
        )
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.relu(x + self.block(x))

# --------- Self-Attention Block ---------
class SelfAttentionBlock(nn.Module):
    def __init__(self, dim, num_heads=4):
        super().__init__()
        self.attn = nn.MultiheadAttention(embed_dim=dim, num_heads=num_heads, batch_first=True)
        self.norm = nn.LayerNorm(dim)

    def forward(self, x):
        x = x.transpose(1, 2)  # (B, C, T) → (B, T, C)
        x2, _ = self.attn(x, x, x)
        x = self.norm(x + x2)
        return x.transpose(1, 2)  # (B, T, C) → (B, C, T)

# --------- UNet1D Encoder ---------
class UNet1DEncoder(nn.Module):
    def __init__(self, in_channels=1, base=32):
        super().__init__()
        self.enc1 = nn.Sequential(
            nn.Conv1d(in_channels, base, 3, padding=1),
            nn.ReLU(),
            ResidualBlock(base)
        )
        self.pool1 = nn.MaxPool1d(2)

        self.enc2 = nn.Sequential(
            nn.Conv1d(base, base*2, 3, padding=1),
            nn.ReLU(),
            ResidualBlock(base*2)
        )
        self.pool2 = nn.MaxPool1d(2)

        self.enc3 = nn.Sequential(
            nn.Conv1d(base*2, base*4, 3, padding=1),
            nn.ReLU(),
            ResidualBlock(base*4),
            SelfAttentionBlock(base*4)
        )
        self.pool3 = nn.MaxPool1d(2)

    def forward(self, x):
        s1 = self.enc1(x)
        s2 = self.enc2(self.pool1(s1))
        s3 = self.enc3(self.pool2(s2))
        z = self.pool3(s3)
        return z, s1, s2, s3

# --------- Center Crop ---------
def center_crop(enc_feature, target_length):
    current_length = enc_feature.shape[-1]
    delta = current_length - target_length
    if delta == 0:
        return enc_feature
    elif delta % 2 == 0:
        return enc_feature[:, :, delta // 2: -delta // 2]
    else:
        return enc_feature[:, :, delta // 2: -(delta // 2) - 1]

# --------- UNet1D Decoder ---------
class UNet1DDecoder(nn.Module):
    def __init__(self, out_channels=1, base=32):
        super().__init__()
        self.up1 = nn.ConvTranspose1d(base*4, base*4, 2, stride=2)
        self.dec1 = nn.Sequential(
            nn.Conv1d(base*8, base*4, 3, padding=1),
            nn.ReLU(),
            ResidualBlock(base*4)
        )

        self.up2 = nn.ConvTranspose1d(base*4, base*2, 2, stride=2)
        self.dec2 = nn.Sequential(
            nn.Conv1d(base*4, base*2, 3, padding=1),
            nn.ReLU(),
            ResidualBlock(base*2)
        )

        self.up3 = nn.ConvTranspose1d(base*2, base, 2, stride=2)
        self.dec3 = nn.Sequential(
            nn.Conv1d(base*2, base, 3, padding=1),
            nn.ReLU(),
            ResidualBlock(base)
        )

        self.final = nn.Conv1d(base, out_channels, 1)

    def forward(self, z, s1, s2, s3):
        x = self.up1(z)
        s3 = center_crop(s3, x.shape[-1])
        x = torch.cat([x, s3], dim=1)
        x = self.dec1(x)

        x = self.up2(x)
        s2 = center_crop(s2, x.shape[-1])
        x = torch.cat([x, s2], dim=1)
        x = self.dec2(x)

        x = self.up3(x)
        s1 = center_crop(s1, x.shape[-1])
        x = torch.cat([x, s1], dim=1)
        x = self.dec3(x)

        return self.final(x)

# --------- CycleUNet Model ---------
class CycleUNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = UNet1DEncoder()
        self.gen_A = UNet1DDecoder()
        self.gen_B = UNet1DDecoder()

    def forward(self, x):
        z, s1, s2, s3 = self.encoder(x)
        x_prime = self.gen_A(z, s1, s2, s3)
        z2, s1_p, s2_p, s3_p = self.encoder(x_prime)
        x_cycle = self.gen_B(z2, s1_p, s2_p, s3_p)
        return x_prime, x_cycle

# --------- Preprocessing ---------
def preprocess_and_split(df):
    df['label'] = df['Hypertension'].apply(lambda x: 0 if str(x).strip().lower() == 'normal' else 1)
    ppg_cols = [str(i) for i in range(1, 2101)]

    unique_ids = df['subject_ID'].unique()
    train_ids, test_ids = train_test_split(unique_ids, test_size=0.4, random_state=42)

    df_train = df[(df['subject_ID'].isin(train_ids)) & (df['label'] == 0)]
    df_test = df[df['subject_ID'].isin(test_ids)]

    scaler = StandardScaler()
    X_train = scaler.fit_transform(df_train[ppg_cols].values)
    X_test = scaler.transform(df_test[ppg_cols].values)

    X_train = torch.tensor(X_train[:, None, :], dtype=torch.float32)
    X_test = torch.tensor(X_test[:, None, :], dtype=torch.float32)
    y_test = df_test['label'].values

    return df_train, df_test, X_train, X_test, y_test

# --------- Training ---------
def train_cycle_unet(X_train, batch_size=32, num_epochs=50, lr=1e-3):
    model = CycleUNet()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    dataset = TensorDataset(X_train)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for batch in loader:
            x = batch[0]
            x_prime, x_cycle = model(x)

            x_crop = center_crop(x, x_prime.shape[-1])

            loss = F.mse_loss(x_crop, x_prime) + F.mse_loss(x_crop, x_cycle)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs} - Loss: {total_loss:.4f}")
    return model

# --------- Evaluation ---------
def test_cycle_unet(model, df_test, X_test, y_test):
    model.eval()
    with torch.no_grad():
        x_prime_pred, x_cycle_pred = model(X_test)
        x_crop = center_crop(X_test, x_cycle_pred.shape[-1])
        scores = F.mse_loss(x_crop, x_cycle_pred, reduction='none').mean(dim=(1, 2)).numpy()

    # 정량화 결과 출력
    df_result = df_test.copy()
    df_result['recon_error'] = scores
    print(df_result.groupby('label')['recon_error'].describe())
    return df_result

# --------- Main ---------
if __name__ == "__main__":
    df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/combined_dataset.xlsx')
    df_train, df_test, X_train, X_test, y_test = preprocess_and_split(df)
    model = train_cycle_unet(X_train)
    result_df = test_cycle_unet(model, df_test, X_test, y_test)


Epoch 1/50 - Loss: 8.3686
Epoch 2/50 - Loss: 1.1793
Epoch 3/50 - Loss: 0.5564
Epoch 4/50 - Loss: 0.4762
Epoch 5/50 - Loss: 0.2132
Epoch 6/50 - Loss: 0.2386
Epoch 7/50 - Loss: 0.2735
Epoch 8/50 - Loss: 0.1446
Epoch 9/50 - Loss: 0.4683
Epoch 10/50 - Loss: 0.1549
Epoch 11/50 - Loss: 0.2160
Epoch 12/50 - Loss: 0.5544
Epoch 13/50 - Loss: 0.1741
Epoch 14/50 - Loss: 0.2506
Epoch 15/50 - Loss: 0.2127
Epoch 16/50 - Loss: 0.1791
Epoch 17/50 - Loss: 0.4950
Epoch 18/50 - Loss: 0.1500
Epoch 19/50 - Loss: 0.1359
Epoch 20/50 - Loss: 0.2192
Epoch 21/50 - Loss: 0.3320
Epoch 22/50 - Loss: 0.4036
Epoch 23/50 - Loss: 0.2083
Epoch 24/50 - Loss: 0.1321
Epoch 25/50 - Loss: 0.1127
Epoch 26/50 - Loss: 0.2697
Epoch 27/50 - Loss: 0.2358
Epoch 28/50 - Loss: 0.1991
Epoch 29/50 - Loss: 0.1027
Epoch 30/50 - Loss: 0.3388
Epoch 31/50 - Loss: 0.2537
Epoch 32/50 - Loss: 0.3687
Epoch 33/50 - Loss: 0.4165
Epoch 34/50 - Loss: 0.4092
Epoch 35/50 - Loss: 0.2693
Epoch 36/50 - Loss: 0.1832
Epoch 37/50 - Loss: 0.2909
Epoch 38/5

In [5]:
pd.set_option('display.max_rows', 1000)
test_cycle_unet(model, df_test, X_test, y_test)

       count      mean       std       min       25%       50%       75%  \
label                                                                      
0       87.0  0.007115  0.004857  0.001516  0.003173  0.005629  0.010036   
1      177.0  0.007564  0.005599  0.001108  0.003471  0.006435  0.009731   

           max  
label           
0      0.02587  
1      0.03339  


,Num.,subject_ID,Sex(M/F),Age(year),Height(cm),Weight(kg),Systolic Blood Pressure(mmHg),Diastolic Blood Pressure(mmHg),Heart Rate(b/m),BMI(kg/m^2),...,2093,2094,2095,2096,2097,2098,2099,2100,label,recon_error
15,6,10,Female,48,160,68,124,62,70,26.562500,...,2114,2112,2112,2109,2109,2109,2083,2083,1,0.001594
16,6,10,Female,48,160,68,124,62,70,26.562500,...,2096,2126,2126,2109,2109,2109,2116,2116,1,0.001698
17,6,10,Female,48,160,68,124,62,70,26.562500,...,2077,2077,2072,2072,2051,2051,2051,2036,1,0.002595
27,10,14,Female,47,150,47,98,56,69,20.888889,...,2220,2220,2220,2234,2234,2240,2240,2240,0,0.009899
28,10,14,Female,47,150,47,98,56,69,20.888889,...,1807,1807,1827,1827,1827,1801,1801,1801,0,0.005561
29,10,14,Female,47,150,47,98,56,69,20.888889,...,1786,1786,1786,1798,1798,1789,1789,1789,0,0.010785
36,13,17,Female,48,155,57,117,70,75,23.725286,...,2107,2107,2120,2120,2139,2139,2139,2119,0,0.003207
37,13,17,Female,48,155,57,117,70,75,23.725286,...,2175,2175,2175,2147,2147,2162,2162,2162,0,0.003672
38,13,17,Female,48,155,57,117,70,75,23.725286,...,2214,2214,2219,2219,2219,2286,2286,2226,0,0.003712
45,16,21,Female,48,150,57,178,86,80,25.333333,...,1956,1956,1956,1936,1936,1962,1962,1962,1,0.005427


In [6]:
result_df

,Num.,subject_ID,Sex(M/F),Age(year),Height(cm),Weight(kg),Systolic Blood Pressure(mmHg),Diastolic Blood Pressure(mmHg),Heart Rate(b/m),BMI(kg/m^2),...,2093,2094,2095,2096,2097,2098,2099,2100,label,recon_error
15,6,10,Female,48,160,68,124,62,70,26.562500,...,2114,2112,2112,2109,2109,2109,2083,2083,1,0.001594
16,6,10,Female,48,160,68,124,62,70,26.562500,...,2096,2126,2126,2109,2109,2109,2116,2116,1,0.001698
17,6,10,Female,48,160,68,124,62,70,26.562500,...,2077,2077,2072,2072,2051,2051,2051,2036,1,0.002595
27,10,14,Female,47,150,47,98,56,69,20.888889,...,2220,2220,2220,2234,2234,2240,2240,2240,0,0.009899
28,10,14,Female,47,150,47,98,56,69,20.888889,...,1807,1807,1827,1827,1827,1801,1801,1801,0,0.005561
29,10,14,Female,47,150,47,98,56,69,20.888889,...,1786,1786,1786,1798,1798,1789,1789,1789,0,0.010785
36,13,17,Female,48,155,57,117,70,75,23.725286,...,2107,2107,2120,2120,2139,2139,2139,2119,0,0.003207
37,13,17,Female,48,155,57,117,70,75,23.725286,...,2175,2175,2175,2147,2147,2162,2162,2162,0,0.003672
38,13,17,Female,48,155,57,117,70,75,23.725286,...,2214,2214,2219,2219,2219,2286,2286,2226,0,0.003712
45,16,21,Female,48,150,57,178,86,80,25.333333,...,1956,1956,1956,1936,1936,1962,1962,1962,1,0.005427


In [8]:
result_df.to_excel("/content/drive/MyDrive/Colab Notebooks/result_df.xlsx", index=False)